In [1]:
import json
import numpy as np
import pandas as pd
import glob2
import torch
import pickle
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import torch.nn as nn
import torch.optim as optim
import pickle as pkl
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import os
from transformers import BertTokenizer, VisualBertModel
import re
import torchvision
resnet = torchvision.models.resnet34(pretrained=True)
resnet=nn.Sequential(*list(resnet.children())[:-1])
import PIL
from PIL import Image
import matplotlib.pyplot as plt

In [131]:
# load data
with open("/mnt/sda/nirmal/rum/code/clustering/sg_memes.json","r") as f:
    data=json.load(f)
stance_memes=[item for item in data['matched_memes']]
len(stance_memes)

1372

In [132]:
# split
pillars=set([pillar for item in stance_memes for pillar in item['pillars']])
pillar_wise={pillar:[] for pillar in pillars}
for item in stance_memes:
    for pillar in item['pillars']:
        pillar_wise[pillar].append(item)
        break
{pillar:len(_) for pillar,_ in pillar_wise.items()}        

{'Social Defence': 28,
 'Digital Defence': 26,
 'Psychological Defence': 254,
 'Economic Defence': 86,
 'Civil Defence': 73,
 'Military Defence': 511,
 'Others': 394}

In [133]:
# 60-20-20 split
train,val,test=[],[],[]
for pillar,items in pillar_wise.items():
    temp=np.random.choice(items,int(0.60*len(items)),replace=False)
    train.extend(temp)
    items=[item for item in items if item["memeId"] not in [meme["memeId"] for meme in train]]
    temp=np.random.choice(items,int(0.5*len(items)),replace=False)
    val.extend(temp)
    items=[item for item in items if item["memeId"] not in [meme["memeId"] for meme in val]]
    test.extend(items)
len(train),len(val),len(test)        

(819, 275, 278)

In [134]:
# train split pillar-wise
train_pillar_distr={pillar:0 for pillar in pillars}
for item in train:
    for pillar in item['pillars']:
        train_pillar_distr[pillar]+=1
        
val_pillar_distr={pillar:0 for pillar in pillars}
for item in val:
    for pillar in item['pillars']:
        val_pillar_distr[pillar]+=1
        
test_pillar_distr={pillar:0 for pillar in pillars}
for item in test:
    for pillar in item['pillars']:
        test_pillar_distr[pillar]+=1   
        
train_pillar_distr,val_pillar_distr,test_pillar_distr        

({'Social Defence': 22,
  'Digital Defence': 15,
  'Psychological Defence': 152,
  'Economic Defence': 71,
  'Civil Defence': 47,
  'Military Defence': 317,
  'Others': 236},
 {'Social Defence': 9,
  'Digital Defence': 7,
  'Psychological Defence': 51,
  'Economic Defence': 21,
  'Civil Defence': 18,
  'Military Defence': 106,
  'Others': 79},
 {'Social Defence': 7,
  'Digital Defence': 6,
  'Psychological Defence': 51,
  'Economic Defence': 23,
  'Civil Defence': 20,
  'Military Defence': 104,
  'Others': 79})

In [135]:
# train upsample
train_others=[item for item in train if 'Others' in item['pillars']]
train_economic=[item for item in train if 'Economic Defence' in item['pillars']]
train_social=[item for item in train if 'Social Defence' in item['pillars']]
train_digital=[item for item in train if 'Digital Defence' in item['pillars']]
train_psychological=[item for item in train if 'Psychological Defence' in item['pillars']]
train_civil=[item for item in train if 'Civil Defence' in item['pillars']]
train_military=[item for item in train if 'Military Defence' in item['pillars']]

train_upsample=[]
train_upsample.extend(train_others)
train_upsample.extend(np.random.choice(train_others,315-236))

train_upsample.extend(train_economic)
train_upsample.extend(np.random.choice(train_economic,315-100))

train_upsample.extend(train_social)
train_upsample.extend(np.random.choice(train_social,315-24)) 

train_upsample.extend(train_digital)
train_upsample.extend(np.random.choice(train_digital,315-17))

train_upsample.extend(train_psychological)
# train_upsample.extend(np.random.choice(train_psychological,315-300))

train_upsample.extend(train_civil)
train_upsample.extend(np.random.choice(train_civil,315-51)) 

train_upsample.extend(train_military)

In [136]:
train_pillar_distr={pillar:0 for pillar in pillars}
for item in train_upsample:
    for pillar in item['pillars']:
        train_pillar_distr[pillar]+=1
train_pillar_distr        

{'Social Defence': 331,
 'Digital Defence': 324,
 'Psychological Defence': 329,
 'Economic Defence': 319,
 'Civil Defence': 360,
 'Military Defence': 404,
 'Others': 315}

In [137]:
# adding image and text info
df=pd.read_csv("/mnt/sda/nirmal/rum/code/clustering/priority_memes.csv")
data={}
image_files=glob2.glob('/mnt/sda/dataset/google_insta_reddit/batch1/postprocessing/with_text/*')
image_files.extend(glob2.glob("/mnt/sda/dataset/google_insta_reddit/batch2/postprocessing/with_text/*"))
image_file_names=[file.split("/")[-1] for file in image_files]
for i,row in df.iterrows():
    data[row['filename']]=(row["text"].replace("\n"," "),image_files[image_file_names.index(row['filename'])])    

In [138]:
# adding text,image path to data
for item in train:
    item.update({'text':data[item['filename']][0],'image_path':data[item['filename']][1]})

for item in val:
    item.update({'text':data[item['filename']][0],'image_path':data[item['filename']][1]})

for item in test:
    item.update({'text':data[item['filename']][0],'image_path':data[item['filename']][1]})

# val=[item.update({'text':data[item['filename']][0],'image_path':data[item['filename']][1]}) for item in val]

# test=[item.update({'text':data[item['filename']][0],'image_path':data[item['filename']][1]}) for item in test]

In [139]:
pillar_labels_dict={pillar:i for i,pillar in enumerate(pillars)}
# print(pillar_labels_dict)
unique_stances=set([stance for item in stance_memes for _,stance in item['stance']])
# # stance_labels_dict={for i,pillar in enumerate(pillars) for indx,stance in enumerate(unique_stances)}
unique_stances={stance:i for i,stance in enumerate(unique_stances)}
stance_labels_dict={}
index=0
for pillar,_ in pillar_labels_dict.items():
    stance_labels_dict[pillar]=[]
    for stance in unique_stances:
        stance_labels_dict[pillar].append(index)
        index+=1
print(pillar_labels_dict,stance_labels_dict)  

{'Social Defence': 0, 'Digital Defence': 1, 'Psychological Defence': 2, 'Economic Defence': 3, 'Civil Defence': 4, 'Military Defence': 5, 'Others': 6} {'Social Defence': [0, 1, 2], 'Digital Defence': [3, 4, 5], 'Psychological Defence': [6, 7, 8], 'Economic Defence': [9, 10, 11], 'Civil Defence': [12, 13, 14], 'Military Defence': [15, 16, 17], 'Others': [18, 19, 20]}


In [208]:
test_imgs=[]
class ClassifierDataset(Dataset):
    def __init__(self,items,embed_typ):
        self.items=items
        self.embed_typ=embed_typ

    def __len__(self): 
        return len(self.items)

    def __getitem__(self,index):
        
        with open("/mnt/sda/nirmal/rum/code/clustering/{}/{}".\
                  format(self.embed_typ,self.items[index]['filename']),"rb") as f:
            encoding=pkl.load(f) 
        
        pillar_labels=torch.zeros(7)  
        for pillar in self.items[index]['pillars']:
            pillar_labels[pillar_labels_dict[pillar]]=1
        stance_labels=torch.zeros(21)  
        for i,stance in enumerate(self.items[index]['stance']):
            if(stance[1]=="Supportive"):
                stance_labels[stance_labels_dict[stance[0]][0]]=1 
            elif(stance[1]=="Neutral"):
                stance_labels[stance_labels_dict[stance[0]][1]]=1
            elif(stance[1]=="Against"):
                stance_labels[stance_labels_dict[stance[0]][2]]=1      
        return encoding,pillar_labels,stance_labels,self.items[index]['filename']

In [203]:
class Pillar_Stance_Classifier(nn.Module):
    def __init__(self,embed_typ):
        super(Pillar_Stance_Classifier, self).__init__()
        if embed_typ=="VGG":
            self.pillar_cls_layer = torch.nn.Sequential(nn.Linear(25088, 4096),nn.ReLU(),nn.Linear(4096, 512),\
                                                       nn.ReLU(),nn.Linear(512,7))#
            self.stance_cls_layer = torch.nn.Sequential(nn.Linear(25088, 4096),nn.ReLU(),nn.Linear(4096, 512),\
                                                       nn.ReLU(),nn.Linear(512,21))
        else:
            self.pillar_cls_layer = torch.nn.Sequential(nn.Linear(1024, 64),nn.ReLU(),nn.Linear(64,7))
            self.stance_cls_layer = torch.nn.Sequential(nn.Linear(1024, 64),nn.ReLU(),nn.Linear(64,21))

    def forward(self,inp):
        output_pillar=self.pillar_cls_layer(inp)
        output_stance=self.stance_cls_layer(inp)
        return output_pillar,output_stance

In [204]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embed_typ="CLIP"
model=Pillar_Stance_Classifier().to(device)
model.train()
train_set=ClassifierDataset(train_upsample,embed_typ=embed_typ)
val_set=ClassifierDataset(val)
train_loader=DataLoader(train_set, batch_size = 64)
val_loader = DataLoader(val_set, batch_size = 64)   
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

In [205]:
val_losses=[]
for _e in range(1000):
    train_loss=0
    for t, (embedding, pillar_labels,stance_labels) in enumerate(train_loader):
        embedding=embedding.to(device)
        
        pillar_labels =pillar_labels.to(device) 
        stance_labels =stance_labels.to(device) 

        optimizer.zero_grad()
        pillar_logits,stance_logits=model(embedding)
        pillar_loss = criterion(pillar_logits.squeeze(1), pillar_labels.float())
        stance_loss = criterion(stance_logits.squeeze(1), stance_labels.float())
        
        loss=pillar_loss+stance_loss
        train_loss+=loss.data.item()
        loss.backward()
        optimizer.step()
    train_loss= np.mean(train_loss)
    val_loss=0
    for t, (embedding,pillar_labels,stance_labels) in enumerate(val_loader):
        embedding=embedding.to(device)
        pillar_labels =pillar_labels.to(device) 
        stance_labels =stance_labels.to(device)  

        optimizer.zero_grad()
        pillar_logits,stance_logits=model(embedding)
        pillar_loss = criterion(pillar_logits.squeeze(1), pillar_labels.float())
        stance_loss = criterion(stance_logits.squeeze(1), stance_labels.float())
        val_loss+=(pillar_loss+stance_loss).data.item()
    val_loss= np.mean(val_loss)   
    if(len(val_losses)>0 and val_loss<min(val_losses)):
          torch.save(model.state_dict(), '{}.pt'.format(embed_typ))  
    val_losses.append(val_loss)      
    print('training loss:{} validation loss:{}'.format(train_loss,val_loss))

training loss:39.54735004901886 validation loss:5.035736918449402
training loss:26.830148458480835 validation loss:3.334825575351715
training loss:21.69740927219391 validation loss:3.026538610458374
training loss:20.841771066188812 validation loss:2.9722471237182617
training loss:20.37072455883026 validation loss:2.9396854043006897
training loss:19.923506498336792 validation loss:2.90233713388443
training loss:19.467327415943146 validation loss:2.8635897636413574
training loss:18.96155270934105 validation loss:2.8170079588890076
training loss:18.446918070316315 validation loss:2.7737756967544556
training loss:17.878470838069916 validation loss:2.7231356501579285
training loss:17.321989744901657 validation loss:2.677002489566803
training loss:16.730896025896072 validation loss:2.6267915964126587
training loss:16.159560680389404 validation loss:2.5827280580997467
training loss:15.569054871797562 validation loss:2.5346655547618866
training loss:15.013923168182373 validation loss:2.4939449

KeyboardInterrupt: 

In [209]:
model=Pillar_Stance_Classifier().to(device)
test_set=ClassifierDataset(test)
test_loader=DataLoader(test_set, batch_size = 1,shuffle=False)
model.load_state_dict(torch.load("model_clip.pt"))

model.eval()
preds=[]
pillar_outputs=[]
stance_outputs=[]
pillar_labels_gt=[]
stance_labels_gt=[]
stance_output_proba=[]
filenames=[]
with torch.no_grad():
    for t, (embedding,pillar_labels,stance_labels,filename) in enumerate(test_loader):
        filenames.append(filename)
        embedding=embedding.to(device)
        pillar_labels =pillar_labels.to(device) 
        stance_labels =stance_labels.to(device)  
        pillar_logits,stance_logits=model(embedding)
        pillar_output,stance_output=torch.sigmoid(pillar_logits),torch.sigmoid(stance_logits)
        temp=[0 if item<0.5 else 1 for item in pillar_output.view(-1).tolist()]

        pillar_outputs.append(temp) 
        stance_outputs.append([0 if item<0.5 else 1 for item in stance_output.view(-1).tolist()])
        stance_output_proba.append(stance_output.view(-1).tolist())
        pillar_labels_gt.append(pillar_labels.view(-1).tolist())
        stance_labels_gt.append(stance_labels.view(-1).tolist())

In [210]:
gt=[[item[n*3:(n+1)*3] for n in range(7)] for item in stance_labels_gt]
stance_outputs=[[item[n*3:(n+1)*3] for n in range(7)] for item in stance_output_proba]

pred_filtered=[]
gt_filtered=[]
for i,item in enumerate(pillar_outputs):
    temp=np.array(item)
    indices=[it for item in np.where(temp==1) for it in item]
    for indx in indices:
#         pred_filtered.append(pred[i][indx])
        arg=np.argmax(stance_outputs[i][indx])
        
        if(arg==0):
            pred_filtered.append(1)
        elif(arg==1):
            pred_filtered.append(2)
        elif(arg==2):
            pred_filtered.append(3)
        if(gt[i][indx]==[0,0,0]):
            gt_filtered.append(0)
        elif(gt[i][indx]==[1,0,0]):
            gt_filtered.append(1)
        elif(gt[i][indx]==[0,1,0]):
            gt_filtered.append(2)
        elif(gt[i][indx]==[0,0,1]):
            gt_filtered.append(3)    

classification_report(gt_filtered,pred_filtered)

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00        32\n           1       0.80      0.27      0.40        15\n           2       0.68      0.92      0.78       106\n           3       0.45      0.51      0.48        39\n\n    accuracy                           0.63       192\n   macro avg       0.48      0.42      0.42       192\nweighted avg       0.53      0.63      0.56       192\n'

In [199]:
accuracy_score(gt_filtered,pred_filtered)

0.5

In [200]:
classification_report(pillar_labels_gt,pillar_outputs)

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00         7\n           1       0.00      0.00      0.00         6\n           2       0.40      0.04      0.07        51\n           3       0.00      0.00      0.00        23\n           4       0.00      0.00      0.00        20\n           5       0.70      0.15      0.25       104\n           6       0.00      0.00      0.00        79\n\n   micro avg       0.64      0.06      0.11       290\n   macro avg       0.16      0.03      0.05       290\nweighted avg       0.32      0.06      0.10       290\n samples avg       0.06      0.06      0.06       290\n'

In [201]:
accuracy_score(pillar_labels_gt,pillar_outputs)

0.06474820143884892